In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
T_p = 1
L = 300
Q_threshold = 40
Q_max = 60
W_max = 500
cycles = 24
M = 3600
episodes = 1000
total_duration = 24 * 3600

W_levels = np.linspace(0, W_max, 26)

#mean_arrival_rates = np.array([80, 60, 40, 30, 40, 50, 70, 100, 75, 70, 80, 90, 100, 90, 100, 120, 110, 160, 170, 180, 200, 160, 130, 90])

In [3]:
def generate_poisson_traffic(λ, duration):
    R = np.random.rand(int(λ * duration * 2))
    IAT = -np.log(1 - R) / λ
    arrivals = np.cumsum(IAT)
    return arrivals[arrivals <= duration]

In [4]:
# #DATA_SET_GENERATION

daily_mean_arrival_counts = np.random.poisson(lam=100, size=episodes)
hourly_mean_arrival_counts = np.zeros((episodes,24))
data_arrival_counts = np.zeros((episodes, total_duration))

for i in range(0, 1100, 1):
  mean_arrival_rates=np.random.poisson(lam=daily_mean_arrival_counts[i], size=24)
  hourly_mean_arrival_counts[i]=mean_arrival_rates
  all_arrivals = []
  for hour, λ in enumerate(mean_arrival_rates):
      arrivals = generate_poisson_traffic(λ, 3600)
      arrivals += hour * 3600
      all_arrivals.extend(arrivals)
  all_arrivals = np.array(all_arrivals)
  time_bins = np.arange(0, total_duration + 1, 1)
  arrival_counts, _ = np.histogram(all_arrivals, bins=time_bins)
  data_arrival_counts[i]=arrival_counts

IndexError: index 1000 is out of bounds for axis 0 with size 1000

In [ ]:
#STORE_IN_CSV_FILES

pd.DataFrame(daily_mean_arrival_counts).to_csv("daily_mean_arrival_counts.csv", index=False, header=["Daily Mean"])
pd.DataFrame(hourly_mean_arrival_counts).to_csv("hourly_mean_arrival_counts.csv", index=False, header=[f"Hour_{i}" for i in range(24)])
pd.DataFrame(data_arrival_counts).to_csv("data_arrival_counts.csv", index=False, header=[f"Second_{i}" for i in range(total_duration)])

print("Files saved successfully!")

In [4]:
# Read daily_mean_arrival_counts (1D array, shape: (1000,))
daily_mean_arrival_counts = pd.read_csv("daily_mean_arrival_counts.csv").values.flatten()
# Read hourly_mean_arrival_counts (2D array, shape: (1000, 24))
hourly_mean_arrival_counts = pd.read_csv("hourly_mean_arrival_counts.csv").values
# Read data_arrival_counts (2D array, shape: (1000, 86400))
data_arrival_counts = pd.read_csv("data_arrival_counts.csv").values

In [9]:
# 1. <50, 2. 50-60, 3. 60-70, 4. 70-80, 5. 80-90, 6. 90-100, 7. 100-110, 8. 110-120, 9. 120-130, 10. 130-140, 11. 140-150, 12. >150
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import tensorflow as tf

class DQNLearningAgent:
    def __init__(self, seed, n_states=15, n_actions=26, gamma=0.5, epsilon=1.0, decay=0.99):
        self.epsilon_min = 0.010
        self.learning_rate = 0.01
        self._state_size = n_states  # 3 queue states + 12 cycle indicators
        self._action_size = n_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay = decay
        self.seed = seed
        random.seed(self.seed)
        np.random.seed(self.seed)
        tf.random.set_seed(self.seed)
        self.model = self._build_model()
        self.qmean_states = np.array([50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150])
        self.qmean_cycle = np.arange(0, 12)
    
    def update_queue(self, Q, new_packets, u, Q_max):
        Q_new=Q+((new_packets-u)*T_p/L)
        return max(0,min(Q_new,Q_max+1))

    def _build_model(self):
        model = Sequential()
        model.add(Dense(2, input_dim=self._state_size, activation='relu'))
        model.add(Dense(2, activation='relu'))
        model.add(Dense(self._action_size, activation='linear'))
        model.compile(loss='mse', optimizer=SGD(learning_rate=self.learning_rate, momentum=0.9, nesterov=True))
        return model

    def _encode_state(self, queue_state, cycle):
        queue_onehot = np.zeros(3)
        queue_onehot[queue_state] = 1.0
        cycle_onehot = np.zeros(12)
        cycle_onehot[cycle] = 1.0
        return np.concatenate([queue_onehot, cycle_onehot])
    
    def _encode_cycle(self,value):
        return self.qmean_cycle[np.searchsorted(self.qmean_states, value)]

    def choose_action(self, q_values):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self._action_size)
        return np.argmax(q_values)

    def update_qlearning(self, state, curr_q_values, action, reward, next_q_values, hour):
        if hour == 23:  
            target = reward
        else:
            target = reward + self.gamma * np.max(next_q_values)
        target_vec = curr_q_values.copy()
        target_vec[action] = target
        self.model.fit(state.reshape(1, -1), target_vec.reshape(1, -1), epochs=1, verbose=0)

    def update_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.decay)
        return self.epsilon


In [10]:
# Initialize agent and data structures
dqn_agent = DQNLearningAgent(seed=42)
dqn_rewards = []
dqn_bandwidths = []

# Training loop
for episode in range(episodes):
    Qlength = 0
    episode_reward = 0
    episode_bandwidth = 0
    queue_state=0
    state = dqn_agent._encode_state(queue_state,dqn_agent._encode_cycle(hourly_mean_arrival_counts[episode][0]))
    curr_q_values = dqn_agent.model.predict(state.reshape(1, -1), verbose=0)[0]

    for cycle in range(24):
        Q_history = []

        # Select action using DQN
        action = dqn_agent.choose_action(curr_q_values)
        u = W_levels[action]
        episode_bandwidth += u

        # Simulate hour of traffic
        for second in range(3600):
            arrivals = data_arrival_counts[episode, cycle*3600 + second]
            Qlength = dqn_agent.update_queue(Qlength, arrivals, u, Q_max)
            Q_history.append(Qlength)


        # Calculate reward
        Q_mean = np.mean(Q_history)
        p_violation = np.mean(np.array(Q_history) >= Q_threshold)
        p_drop = np.mean(np.array(Q_history) >= Q_max)

        if p_violation <= 0.1 and p_drop <= 0.01:
            reward = Q_mean + (1 / u) if u != 0 else Q_mean
        elif (p_violation <= 0.1 and p_drop > 0.01) or (p_violation > 0.1 and p_drop <= 0.01):
            reward = 0
        else:
            reward = -1

        # Determine next state
        if Qlength <= Q_threshold:
            next_queue_state = 0
        elif Qlength < Q_max:
            next_queue_state = 1
        else:
            next_queue_state = 2


        if(cycle!=23):
            next_state = dqn_agent._encode_state(next_queue_state, dqn_agent._encode_cycle(hourly_mean_arrival_counts[episode][cycle+1]))
            next_q_values = dqn_agent.model.predict(next_state.reshape(1, -1), verbose=0)[0]
        else:
            next_state=0
            next_q_values=0

        # Update Q-network
        dqn_agent.update_qlearning(state, curr_q_values, action, reward, next_q_values, cycle)
        episode_reward += reward

        queue_state=next_queue_state
        state=next_state
        curr_q_values=next_q_values
        #print(f"Bandwidth: {u}, Qmean: {hourly_mean_arrival_counts[episode, cycle]}")

    # Update exploration rate
    dqn_agent.update_epsilon()

    # Store episode results
    dqn_rewards.append(episode_reward)
    dqn_bandwidths.append(episode_bandwidth)

    # Optional: Print progress
    print(f"Episode {episode+1}, Avg Reward: {np.mean(dqn_rewards[-100:])}, Bandwidth: {episode_bandwidth}")

print("Training completed!")

Episode 1, Avg Reward: -4.02830041902542, Bandwidth: 5560.0
Episode 2, Avg Reward: -2.431689291752816, Bandwidth: 5840.0
Episode 3, Avg Reward: -1.123484677095509, Bandwidth: 5700.0
Episode 4, Avg Reward: 2.5404220332174434, Bandwidth: 6300.0
Episode 5, Avg Reward: 1.07917109293669, Bandwidth: 6300.0
Episode 6, Avg Reward: 0.16978729140527538, Bandwidth: 4900.0
Episode 7, Avg Reward: 0.6406938595891046, Bandwidth: 4880.0
Episode 8, Avg Reward: 0.19553228828819735, Bandwidth: 6780.0
Episode 9, Avg Reward: -0.09951653143350124, Bandwidth: 5820.0
Episode 10, Avg Reward: -0.20226763056484792, Bandwidth: 6000.0
Episode 11, Avg Reward: -0.44705795095491613, Bandwidth: 7020.0
Episode 12, Avg Reward: -0.5744278004975676, Bandwidth: 5020.0
Episode 13, Avg Reward: -0.8179301710173091, Bandwidth: 6000.0
Episode 14, Avg Reward: -0.8798393962232512, Bandwidth: 5400.0
Episode 15, Avg Reward: -0.8081696619126323, Bandwidth: 5560.0
Episode 16, Avg Reward: -0.699380284103785, Bandwidth: 5500.0
Episode 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Parameters
window_size = 10  # Number of past days used to predict the next day
episodes = 1000  # Total number of days in the dataset

# Assuming hourly_mean_arrival_counts is generated and is a 1000x24 array
# hourly_mean_arrival_counts = np.load(...) or from the given code

# Generate input sequences and targets
X, y = [], []
for i in range(len(hourly_mean_arrival_counts) - window_size):
    X.append(hourly_mean_arrival_counts[i:i+window_size])
    y.append(hourly_mean_arrival_counts[i+window_size])
X = np.array(X)
y = np.array(y)

# Split into training and testing sets
test_start_idx = 800 - window_size  # Test set starts at day 800
train_X, train_y = X[:test_start_idx], y[:test_start_idx]
test_X, test_y = X[test_start_idx:], y[test_start_idx:]

# Normalize the data
scaler = MinMaxScaler()
# Reshape training data for fitting the scaler
train_X_reshaped = train_X.reshape(-1, 24)
scaler.fit(train_X_reshaped)

# Scale training and test data
scaled_train_X = scaler.transform(train_X_reshaped).reshape(train_X.shape)
scaled_test_X = scaler.transform(test_X.reshape(-1, 24)).reshape(test_X.shape)
scaled_train_y = scaler.transform(train_y.reshape(-1, 24)).reshape(train_y.shape)
scaled_test_y = scaler.transform(test_y.reshape(-1, 24)).reshape(test_y.shape)

# Build LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(window_size, 24)))
model.add(Dense(24))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(
    scaled_train_X, scaled_train_y,
    epochs=100,
    batch_size=32,
    validation_data=(scaled_test_X, scaled_test_y),
    verbose=1
)

# Predict on test data
scaled_predictions = model.predict(scaled_test_X)
predictions = scaler.inverse_transform(scaled_predictions)

# Evaluate predictions
mse = np.mean((test_y - predictions) ** 2)
mae = np.mean(np.abs(test_y - predictions))
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")

# Example prediction for the next day (using the last window of test data)
last_window = scaled_test_X[-1].reshape(1, window_size, 24)
predicted_next_day_scaled = model.predict(last_window)
predicted_next_day = scaler.inverse_transform(predicted_next_day_scaled)
print("Predicted Hourly Means for Next Day:", predicted_next_day)

Epoch 1/100
25/25 [==============================] - 2s 24ms/step - loss: 0.1328 - val_loss: 0.0368
Epoch 2/100
25/25 [==============================] - 0s 13ms/step - loss: 0.0315 - val_loss: 0.0296
Epoch 3/100
25/25 [==============================] - 0s 13ms/step - loss: 0.0289 - val_loss: 0.0279
Epoch 4/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0283 - val_loss: 0.0279
Epoch 5/100
25/25 [==============================] - 0s 14ms/step - loss: 0.0280 - val_loss: 0.0274
Epoch 6/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0275 - val_loss: 0.0275
Epoch 7/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0277 - val_loss: 0.0271
Epoch 8/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0273 - val_loss: 0.0269
Epoch 9/100
25/25 [==============================] - 0s 9ms/step - loss: 0.0273 - val_loss: 0.0268
Epoch 10/100
25/25 [==============================] - 0s 9ms/step - loss: 0.0271 - val_loss: 0.0268
Epo

In [25]:
#TEST DATA

# Generate test daily and hourly means
test_daily_mean_arrival_counts = np.random.poisson(lam=100, size=1)  # Simulated daily mean queue
test_hourly_mean_arrival_counts = np.zeros((1,24))  # Placeholder for hourly means
test_data_arrival_counts = np.zeros((1, total_duration))  # Placeholder for arrivals

# Generate hourly mean arrival rates
test_hourly_mean_arrival_counts = np.random.poisson(lam=test_daily_mean_arrival_counts[0], size=24)
#test_hourly_mean_arrival_rates = np.array([[80, 60, 40, 30, 40, 50, 70, 100, 75, 70, 80, 90, 100, 90, 100, 120, 110, 160, 170, 180, 200, 160, 130, 90]])

# Generate Poisson arrivals for each hour
all_arrivals = []
for hour, λ in enumerate(test_hourly_mean_arrival_counts):
    arrivals = generate_poisson_traffic(λ, 3600)
    arrivals += hour * 3600  # Adjust timestamps
    all_arrivals.extend(arrivals)

# Convert arrival timestamps into a histogram (counts per second)
all_arrivals = np.array(all_arrivals)
time_bins = np.arange(0, total_duration + 1, 1)
arrival_counts, _ = np.histogram(all_arrivals, bins=time_bins)
test_data_arrival_counts[0, :] = arrival_counts  # Store test data

print("Test data generated successfully!")
print(test_hourly_mean_arrival_counts)

test_rewards = []
test_bandwidths = []

for episode in range(1):  # Running a single test episode
    Qlength = 0
    episode_reward = 0
    episode_bandwidth = 0
    queue_state = 0

    state = dqn_agent._encode_state(queue_state, dqn_agent._encode_cycle(test_hourly_mean_arrival_counts[0]))
    curr_q_values = dqn_agent.model.predict(state.reshape(1, -1), verbose=0)[0]

    for cycle in range(24):
        Q_history = []

        # Select action using DQN
        action = dqn_agent.choose_action(curr_q_values)
        u = W_levels[action]
        episode_bandwidth += u

        # Simulate queue updates over an hour
        for second in range(3600):
            arrivals = test_data_arrival_counts[0, cycle * 3600 + second]
            Qlength = dqn_agent.update_queue(Qlength, arrivals, u, Q_max)
            Q_history.append(Qlength)

        qmean_curr_state = dqn_agent._encode_cycle(test_hourly_mean_arrival_counts[cycle])
        
        # Calculate reward
        Q_mean = np.mean(Q_history)
        p_violation = np.mean(np.array(Q_history) >= Q_threshold)
        p_drop = np.mean(np.array(Q_history) >= Q_max)

        if p_violation <= 0.1 and p_drop <= 0.01:
            #print("YES")
            reward = Q_mean + (1 / u) if u != 0 else Q_mean
        elif (p_violation <= 0.1 and p_drop > 0.01) or (p_violation > 0.1 and p_drop <= 0.01):
            print("DAMM")
            reward = 0
        else:
            print("OH NO")
            reward = -1

        # Determine next state
        if Qlength <= Q_threshold:
            next_queue_state = 0
        elif Qlength < Q_max:
            next_queue_state = 1
        else:
            next_queue_state = 2

        # Get next state
        if cycle != 23:
            next_state = dqn_agent._encode_state(next_queue_state, dqn_agent._encode_cycle(test_hourly_mean_arrival_counts[cycle + 1]))
            next_q_values = dqn_agent.model.predict(next_state.reshape(1, -1), verbose=0)[0]
        else:
            next_state = 0
            next_q_values = 0
        episode_reward += reward

        queue_state = next_queue_state
        state = next_state
        curr_q_values = next_q_values
        print(cycle,test_hourly_mean_arrival_counts[cycle],u)

    # Store test episode results
    test_rewards.append(episode_reward)
    test_bandwidths.append(episode_bandwidth)

    print(f"Test Episode, Reward: {episode_reward}, Bandwidth: {episode_bandwidth}")
    print(sum(hourly_mean_arrival_counts[0]))

print("Testing completed!")


Test data generated successfully!
[ 98 100  84  98  99  74 106  99  95  89  84  83 106  97 108  85  95  97
  94  93 103  93  97 117]
0 98 120.0
1 100 120.0
2 84 120.0
3 98 120.0
4 99 120.0
5 74 120.0
OH NO
6 106 100.0
7 99 120.0
8 95 120.0
9 89 120.0
10 84 120.0
11 83 120.0
OH NO
12 106 100.0
13 97 120.0
OH NO
14 108 100.0
15 85 120.0
16 95 120.0
17 97 120.0
18 94 120.0
19 93 120.0
20 103 100.0
21 93 120.0
22 97 120.0
23 117 120.0
Test Episode, Reward: 33.090290740740734, Bandwidth: 2800.0
2566.0
Testing completed!
